In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

BASE_PATH = "/content/drive/MyDrive/Enterprise_Fraud_Risk_Reporting"
os.makedirs(BASE_PATH, exist_ok=True)
os.chdir(BASE_PATH)

print("Working directory:", os.getcwd())

Working directory: /content/drive/MyDrive/Enterprise_Fraud_Risk_Reporting


In [3]:
folders = [
    "data_raw",
    "data_processed",
    "sql",
    "notebooks",
    "outputs"
]

for f in folders:
    os.makedirs(f, exist_ok=True)

print("✅ Project folders created")

✅ Project folders created


In [4]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

np.random.seed(7)

# -------------------------
# CONFIG
# -------------------------
n_txn = 250_000
start_date = datetime(2025, 1, 1)
end_date   = datetime(2025, 12, 1)
days = (end_date - start_date).days

channels = ["Card Present", "Card Not Present", "ATM", "Digital Wallet"]
products = ["Classic", "Rewards", "Premium", "Secured"]
lob = ["Consumer", "Small Business"]
merchant_cat = ["Grocery", "Fuel", "Electronics", "Travel", "Dining", "Online Retail", "Other"]

# -------------------------
# TRANSACTIONS
# -------------------------
dates = [start_date + timedelta(days=int(x)) for x in np.random.randint(0, days, size=n_txn)]
amount = np.round(np.random.lognormal(mean=3.3, sigma=0.75, size=n_txn), 2)  # skewed spend
amount = np.clip(amount, 1, 5000)

df = pd.DataFrame({
    "txn_id": np.arange(1, n_txn+1),
    "txn_ts": pd.to_datetime(dates) + pd.to_timedelta(np.random.randint(0, 86400, size=n_txn), unit="s"),
    "customer_id": np.random.randint(100000, 200000, size=n_txn),
    "account_id": np.random.randint(500000, 650000, size=n_txn),
    "lob": np.random.choice(lob, size=n_txn, p=[0.82, 0.18]),
    "product": np.random.choice(products, size=n_txn, p=[0.45, 0.30, 0.18, 0.07]),
    "channel": np.random.choice(channels, size=n_txn, p=[0.28, 0.46, 0.08, 0.18]),
    "mcc_group": np.random.choice(merchant_cat, size=n_txn, p=[0.15, 0.10, 0.08, 0.07, 0.10, 0.35, 0.15]),
    "state": np.random.choice(
        ["CA","TX","FL","NY","PA","NJ","IL","OH","GA","NC","MI","VA","MA","WA","AZ","TN","IN","MO","MD","CO"],
        size=n_txn
    ),
    "amount": amount
})

# Fraud risk signal features (synthetic)
df["is_international"] = np.random.binomial(1, 0.06, size=n_txn)
df["is_new_payee"] = np.random.binomial(1, 0.10, size=n_txn)
df["device_change"] = np.random.binomial(1, 0.08, size=n_txn)
df["velocity_1h"] = np.random.poisson(lam=1.2, size=n_txn)  # count of txns last hour (synthetic)
df["risk_score"] = np.clip(
    15
    + 18*df["is_international"]
    + 10*df["is_new_payee"]
    + 12*df["device_change"]
    + 6*np.minimum(df["velocity_1h"], 5)
    + 0.004*df["amount"]
    + np.random.normal(0, 7, size=n_txn),
    0, 100
)

# Fraud label: higher probability for CNP, international, device changes, high scores
base = 0.007
p = (
    base
    + 0.010*(df["channel"]=="Card Not Present").astype(int)
    + 0.006*(df["channel"]=="Digital Wallet").astype(int)
    + 0.010*df["is_international"]
    + 0.006*df["device_change"]
    + 0.00008*np.maximum(df["risk_score"]-40, 0)
)
p = np.clip(p, 0, 0.35)
df["is_fraud"] = (np.random.rand(n_txn) < p).astype(int)

# Financial impact fields
df["fraud_loss_amt"] = np.where(df["is_fraud"]==1, df["amount"], 0.0)
df["recovery_amt"] = np.where(df["is_fraud"]==1, np.round(df["fraud_loss_amt"]*np.random.uniform(0.05, 0.55, size=n_txn),2), 0.0)

# Decision/ops fields
df["decision"] = np.where(df["risk_score"] >= 65, "Decline", "Approve")
df["was_alerted"] = np.where(df["risk_score"] >= 70, 1, 0)

# -------------------------
# CASES (Ops queue style)
# -------------------------
fraud_txns = df[df["is_fraud"]==1].sample(frac=0.70, random_state=7)  # not all fraud becomes a case
case_ids = np.arange(1, len(fraud_txns)+1)

cases = pd.DataFrame({
    "case_id": case_ids,
    "opened_ts": fraud_txns["txn_ts"].values,
    "txn_id": fraud_txns["txn_id"].values,
    "queue": np.random.choice(["High Risk", "CNP Review", "ATM Review", "Wallet Review"], size=len(fraud_txns), p=[0.35,0.40,0.10,0.15]),
    "status": np.random.choice(["Open", "In Review", "Closed"], size=len(fraud_txns), p=[0.12,0.18,0.70]),
    "disposition": np.random.choice(["Confirmed Fraud", "Not Fraud", "Customer Error"], size=len(fraud_txns), p=[0.70,0.22,0.08])
})
cases["closed_ts"] = pd.to_datetime(cases["opened_ts"]) + pd.to_timedelta(np.random.randint(2, 240, size=len(cases)), unit="h")

# -------------------------
# GL SUMMARY (Finance view)
# -------------------------
df["txn_date"] = df["txn_ts"].dt.date
gl = df.groupby(["txn_date","lob"]).agg(
    fraud_loss=("fraud_loss_amt","sum"),
    recoveries=("recovery_amt","sum"),
    txn_cnt=("txn_id","count"),
    fraud_cnt=("is_fraud","sum")
).reset_index()
gl["net_loss"] = gl["fraud_loss"] - gl["recoveries"]

# -------------------------
# RULES / ALERTS (Monitoring)
# -------------------------
alerts = df[df["was_alerted"]==1].copy()
alerts = alerts.sample(n=min(30_000, len(alerts)), random_state=7)
alerts["rule_name"] = np.where(alerts["risk_score"]>=85, "Rule-HighRiskScore85",
                        np.where(alerts["is_international"]==1, "Rule-International",
                        np.where(alerts["velocity_1h"]>=4, "Rule-Velocity4Plus", "Rule-Score70Plus")))

# -------------------------
# OUTPUT
# -------------------------
import os
os.makedirs("data_raw", exist_ok=True)

df.drop(columns=["txn_date"]).to_csv("data_raw/transactions.csv", index=False)
cases.to_csv("data_raw/cases.csv", index=False)
gl.to_csv("data_raw/gl_summary.csv", index=False)
alerts[["txn_id","txn_ts","account_id","risk_score","rule_name","amount","is_fraud"]].to_csv("data_raw/rules_alerts.csv", index=False)

print("✅ Generated:")
print(" - data_raw/transactions.csv")
print(" - data_raw/cases.csv")
print(" - data_raw/gl_summary.csv")
print(" - data_raw/rules_alerts.csv")

✅ Generated:
 - data_raw/transactions.csv
 - data_raw/cases.csv
 - data_raw/gl_summary.csv
 - data_raw/rules_alerts.csv


In [5]:
import pandas as pd

pd.read_csv("data_raw/transactions.csv").head()

,txn_id,txn_ts,customer_id,account_id,lob,product,channel,mcc_group,state,amount,is_international,is_new_payee,device_change,velocity_1h,risk_score,is_fraud,fraud_loss_amt,recovery_amt,decision,was_alerted
0,1,2025-06-25 03:54:25,118351,601226,Consumer,Premium,Digital Wallet,Other,CO,25.21,0,1,0,1,35.792676,0,0.0,0.0,Approve,0
1,2,2025-07-16 05:48:50,153328,575997,Consumer,Classic,Digital Wallet,Electronics,AZ,27.38,0,0,0,2,21.181732,0,0.0,0.0,Approve,0
2,3,2025-01-26 19:59:40,158764,582044,Consumer,Rewards,ATM,Other,AZ,62.67,0,0,0,0,16.810732,0,0.0,0.0,Approve,0
3,4,2025-03-09 22:56:49,193368,569663,Consumer,Classic,Digital Wallet,Dining,CO,14.06,0,0,0,2,29.156452,0,0.0,0.0,Approve,0
4,5,2025-07-31 07:04:09,195060,638149,Consumer,Classic,Card Not Present,Online Retail,FL,6.40,0,0,0,1,21.219279,0,0.0,0.0,Approve,0


In [ ]:
# to downlad files

from google.colab import files
files.download("data_raw/transactions.csv")

## Install + set up DuckDB (SQL on your CSVs)

In [6]:
!pip -q install duckdb

In [7]:
import duckdb
import pandas as pd
import os

con = duckdb.connect(database=':memory:')

# Confirm files exist
for f in ["data_raw/transactions.csv", "data_raw/cases.csv", "data_raw/gl_summary.csv", "data_raw/rules_alerts.csv"]:
    print(f, "✅" if os.path.exists(f) else "❌")

# Create SQL views over the CSVs (no loading needed)
con.execute("""
CREATE OR REPLACE VIEW transactions AS
SELECT * FROM read_csv_auto('data_raw/transactions.csv');
""")

con.execute("""
CREATE OR REPLACE VIEW cases AS
SELECT * FROM read_csv_auto('data_raw/cases.csv');
""")

con.execute("""
CREATE OR REPLACE VIEW gl_summary AS
SELECT * FROM read_csv_auto('data_raw/gl_summary.csv');
""")

con.execute("""
CREATE OR REPLACE VIEW rules_alerts AS
SELECT * FROM read_csv_auto('data_raw/rules_alerts.csv');
""")

print("✅ DuckDB views created: transactions, cases, gl_summary, rules_alerts")

data_raw/transactions.csv ✅
data_raw/cases.csv ✅
data_raw/gl_summary.csv ✅
data_raw/rules_alerts.csv ✅
✅ DuckDB views created: transactions, cases, gl_summary, rules_alerts


## Executive KPI Pack (exports CSVs to outputs/)

In [8]:
# Daily exec KPIs (fraud rate, net loss, approvals/declines)

os.makedirs("outputs", exist_ok=True)

daily_exec = con.execute("""
WITH base AS (
  SELECT
    CAST(txn_ts AS DATE) AS dt,
    lob,
    COUNT(*) AS txn_cnt,
    SUM(is_fraud) AS fraud_cnt,
    SUM(fraud_loss_amt) AS fraud_loss,
    SUM(recovery_amt) AS recoveries,
    SUM(CASE WHEN decision='Decline' THEN 1 ELSE 0 END) AS declines,
    SUM(CASE WHEN decision='Approve' THEN 1 ELSE 0 END) AS approves
  FROM transactions
  GROUP BY 1,2
)
SELECT
  dt,
  lob,
  txn_cnt,
  fraud_cnt,
  ROUND(100.0 * fraud_cnt / NULLIF(txn_cnt,0), 3) AS fraud_rate_pct,
  ROUND(fraud_loss, 2) AS fraud_loss,
  ROUND(recoveries, 2) AS recoveries,
  ROUND(fraud_loss - recoveries, 2) AS net_loss,
  declines,
  approves,
  ROUND(100.0 * declines / NULLIF(txn_cnt,0), 2) AS decline_rate_pct
FROM base
ORDER BY dt, lob;
""").df()

daily_exec.to_csv("outputs/daily_exec_kpis.csv", index=False)
daily_exec.head(10)

,dt,lob,txn_cnt,fraud_cnt,fraud_rate_pct,fraud_loss,recoveries,net_loss,declines,approves,decline_rate_pct
0,2025-01-01,Consumer,654,10.0,1.529,285.88,99.20,186.68,3.0,651.0,0.46
1,2025-01-01,Small Business,118,5.0,4.237,78.47,19.82,58.65,0.0,118.0,0.00
2,2025-01-02,Consumer,617,11.0,1.783,429.10,109.65,319.45,2.0,615.0,0.32
3,2025-01-02,Small Business,125,5.0,4.000,145.48,46.80,98.68,0.0,125.0,0.00
4,2025-01-03,Consumer,597,6.0,1.005,346.13,107.63,238.50,1.0,596.0,0.17
5,2025-01-03,Small Business,133,2.0,1.504,179.89,56.74,123.15,0.0,133.0,0.00
6,2025-01-04,Consumer,639,8.0,1.252,306.43,105.69,200.74,1.0,638.0,0.16
7,2025-01-04,Small Business,151,3.0,1.987,77.25,29.71,47.54,0.0,151.0,0.00
8,2025-01-05,Consumer,615,7.0,1.138,154.04,46.64,107.40,2.0,613.0,0.33
9,2025-01-05,Small Business,122,2.0,1.639,54.94,26.00,28.94,1.0,121.0,0.82


In [9]:
# Channel / Product contribution (what’s driving risk)

channel_product = con.execute("""
SELECT
  channel,
  product,
  COUNT(*) AS txn_cnt,
  SUM(is_fraud) AS fraud_cnt,
  ROUND(100.0 * SUM(is_fraud) / NULLIF(COUNT(*),0), 3) AS fraud_rate_pct,
  ROUND(SUM(fraud_loss_amt), 2) AS fraud_loss,
  ROUND(SUM(recovery_amt), 2) AS recoveries,
  ROUND(SUM(fraud_loss_amt) - SUM(recovery_amt), 2) AS net_loss
FROM transactions
GROUP BY 1,2
ORDER BY net_loss DESC, fraud_loss DESC
LIMIT 30;
""").df()

channel_product.to_csv("outputs/channel_product_top_drivers.csv", index=False)
channel_product


,channel,product,txn_cnt,fraud_cnt,fraud_rate_pct,fraud_loss,recoveries,net_loss
0,Card Not Present,Classic,51807,962.0,1.857,35781.36,10989.90,24791.46
1,Card Not Present,Rewards,34386,645.0,1.876,22788.63,7125.87,15662.76
2,Card Not Present,Premium,20908,399.0,1.908,14368.38,4227.16,10141.22
3,Digital Wallet,Classic,20255,294.0,1.451,11261.11,3407.20,7853.91
4,Card Present,Classic,31439,244.0,0.776,8457.61,2356.79,6100.82
5,Digital Wallet,Rewards,13610,183.0,1.345,6944.98,2436.53,4508.45
6,Card Present,Rewards,20874,156.0,0.747,5486.02,1510.61,3975.41
7,Card Not Present,Secured,8028,153.0,1.906,5625.70,1786.32,3839.38
8,Digital Wallet,Premium,8273,117.0,1.414,4918.58,1333.55,3585.03
9,Card Present,Premium,12559,118.0,0.940,4618.55,1459.60,3158.95


In [10]:
# “False positive” proxy (declines that weren’t fraud)

false_positive_proxy = con.execute("""
WITH d AS (
  SELECT
    CAST(txn_ts AS DATE) AS dt,
    COUNT(*) AS txn_cnt,
    SUM(CASE WHEN decision='Decline' THEN 1 ELSE 0 END) AS declines,
    SUM(CASE WHEN decision='Decline' AND is_fraud=0 THEN 1 ELSE 0 END) AS declines_nonfraud,
    SUM(CASE WHEN decision='Decline' AND is_fraud=1 THEN 1 ELSE 0 END) AS declines_fraud
  FROM transactions
  GROUP BY 1
)
SELECT
  dt,
  txn_cnt,
  declines,
  declines_fraud,
  declines_nonfraud,
  ROUND(100.0 * declines_nonfraud / NULLIF(declines,0), 2) AS fp_rate_within_declines_pct
FROM d
ORDER BY dt;
""").df()

false_positive_proxy.to_csv("outputs/false_positive_proxy_daily.csv", index=False)
false_positive_proxy.tail(10)

,dt,txn_cnt,declines,declines_fraud,declines_nonfraud,fp_rate_within_declines_pct
324,2025-11-21,760,1.0,0.0,1.0,100.00
325,2025-11-22,712,2.0,0.0,2.0,100.00
326,2025-11-23,761,6.0,1.0,5.0,83.33
327,2025-11-24,754,2.0,0.0,2.0,100.00
328,2025-11-25,763,1.0,0.0,1.0,100.00
329,2025-11-26,757,1.0,0.0,1.0,100.00
330,2025-11-27,748,2.0,0.0,2.0,100.00
331,2025-11-28,755,1.0,0.0,1.0,100.00
332,2025-11-29,743,0.0,0.0,0.0,NaN
333,2025-11-30,708,0.0,0.0,0.0,NaN


In [11]:
# Finance/GL reconciliation view (Ops ↔ GL controls)

recon = con.execute("""
WITH ops AS (
  SELECT
    CAST(txn_ts AS DATE) AS dt,
    lob,
    ROUND(SUM(fraud_loss_amt), 2) AS ops_fraud_loss,
    ROUND(SUM(recovery_amt), 2) AS ops_recoveries,
    ROUND(SUM(fraud_loss_amt) - SUM(recovery_amt), 2) AS ops_net_loss
  FROM transactions
  GROUP BY 1,2
),
fin AS (
  SELECT
    CAST(txn_date AS DATE) AS dt,
    lob,
    ROUND(SUM(fraud_loss), 2) AS gl_fraud_loss,
    ROUND(SUM(recoveries), 2) AS gl_recoveries,
    ROUND(SUM(net_loss), 2) AS gl_net_loss
  FROM gl_summary
  GROUP BY 1,2
)
SELECT
  o.dt,
  o.lob,
  o.ops_fraud_loss, f.gl_fraud_loss, ROUND(o.ops_fraud_loss - f.gl_fraud_loss, 2) AS diff_fraud_loss,
  o.ops_recoveries, f.gl_recoveries, ROUND(o.ops_recoveries - f.gl_recoveries, 2) AS diff_recoveries,
  o.ops_net_loss, f.gl_net_loss, ROUND(o.ops_net_loss - f.gl_net_loss, 2) AS diff_net_loss
FROM ops o
JOIN fin f USING (dt, lob)
ORDER BY o.dt, o.lob;
""").df()

recon.to_csv("outputs/gl_reconciliation_daily.csv", index=False)
recon.head(10)

,dt,lob,ops_fraud_loss,gl_fraud_loss,diff_fraud_loss,ops_recoveries,gl_recoveries,diff_recoveries,ops_net_loss,gl_net_loss,diff_net_loss
0,2025-01-01,Consumer,285.88,285.88,0.0,99.20,99.20,0.0,186.68,186.68,0.0
1,2025-01-01,Small Business,78.47,78.47,0.0,19.82,19.82,0.0,58.65,58.65,0.0
2,2025-01-02,Consumer,429.10,429.10,0.0,109.65,109.65,0.0,319.45,319.45,0.0
3,2025-01-02,Small Business,145.48,145.48,0.0,46.80,46.80,0.0,98.68,98.68,0.0
4,2025-01-03,Consumer,346.13,346.13,0.0,107.63,107.63,0.0,238.50,238.50,0.0
5,2025-01-03,Small Business,179.89,179.89,0.0,56.74,56.74,0.0,123.15,123.15,0.0
6,2025-01-04,Consumer,306.43,306.43,0.0,105.69,105.69,0.0,200.74,200.74,0.0
7,2025-01-04,Small Business,77.25,77.25,0.0,29.71,29.71,0.0,47.54,47.54,0.0
8,2025-01-05,Consumer,154.04,154.04,0.0,46.64,46.64,0.0,107.40,107.40,0.0
9,2025-01-05,Small Business,54.94,54.94,0.0,26.00,26.00,0.0,28.94,28.94,0.0


## Trend Monitoring: “What changed?” weekly signal report

In [12]:
weekly_signals = con.execute("""
WITH w AS (
  SELECT
    DATE_TRUNC('week', CAST(txn_ts AS DATE)) AS week_start,
    lob,
    COUNT(*) AS txn_cnt,
    SUM(is_fraud) AS fraud_cnt,
    SUM(fraud_loss_amt) AS fraud_loss,
    SUM(recovery_amt) AS recoveries
  FROM transactions
  GROUP BY 1,2
),
calc AS (
  SELECT
    week_start,
    lob,
    txn_cnt,
    fraud_cnt,
    ROUND(100.0 * fraud_cnt / NULLIF(txn_cnt,0), 3) AS fraud_rate_pct,
    ROUND(fraud_loss - recoveries, 2) AS net_loss
  FROM w
),
lagged AS (
  SELECT
    *,
    LAG(fraud_rate_pct) OVER (PARTITION BY lob ORDER BY week_start) AS prev_fraud_rate_pct,
    LAG(net_loss) OVER (PARTITION BY lob ORDER BY week_start) AS prev_net_loss
  FROM calc
)
SELECT
  week_start,
  lob,
  txn_cnt,
  fraud_cnt,
  fraud_rate_pct,
  ROUND(fraud_rate_pct - prev_fraud_rate_pct, 3) AS wow_fraud_rate_delta_pct,
  net_loss,
  ROUND(net_loss - prev_net_loss, 2) AS wow_net_loss_delta,
  CASE
    WHEN prev_net_loss IS NOT NULL AND (net_loss - prev_net_loss) > 50000 THEN '🚨 Net Loss Spike'
    WHEN prev_fraud_rate_pct IS NOT NULL AND (fraud_rate_pct - prev_fraud_rate_pct) > 0.25 THEN '⚠️ Fraud Rate Increase'
    ELSE 'OK'
  END AS signal
FROM lagged
ORDER BY week_start, lob;
""").df()

weekly_signals.to_csv("outputs/weekly_signals_exec.csv", index=False)
weekly_signals.tail(12)

,week_start,lob,txn_cnt,fraud_cnt,fraud_rate_pct,wow_fraud_rate_delta_pct,net_loss,wow_net_loss_delta,signal
84,2025-10-20,Consumer,4385,53.0,1.209,-0.472,1567.75,-384.75,OK
85,2025-10-20,Small Business,935,15.0,1.604,-0.616,395.53,-180.40,OK
86,2025-10-27,Consumer,4208,65.0,1.545,0.336,1970.05,402.30,⚠️ Fraud Rate Increase
87,2025-10-27,Small Business,958,12.0,1.253,-0.351,348.88,-46.65,OK
88,2025-11-03,Consumer,4404,64.0,1.453,-0.092,1847.15,-122.90,OK
89,2025-11-03,Small Business,942,9.0,0.955,-0.298,271.15,-77.73,OK
90,2025-11-10,Consumer,4299,75.0,1.745,0.292,1780.26,-66.89,⚠️ Fraud Rate Increase
91,2025-11-10,Small Business,925,15.0,1.622,0.667,398.03,126.88,⚠️ Fraud Rate Increase
92,2025-11-17,Consumer,4196,61.0,1.454,-0.291,1455.36,-324.90,OK
93,2025-11-17,Small Business,984,20.0,2.033,0.411,449.04,51.01,⚠️ Fraud Rate Increase


## Auto-generate an Executive Summary (PowerPoint-ready text)

In [13]:
# Pick the most recent date in daily KPIs
latest_dt = pd.to_datetime(daily_exec["dt"]).max()
latest = daily_exec[pd.to_datetime(daily_exec["dt"]) == latest_dt].copy()

# Overall rollups on that day
day_txn = int(latest["txn_cnt"].sum())
day_fraud = int(latest["fraud_cnt"].sum())
day_rate = (day_fraud / day_txn * 100) if day_txn else 0
day_net_loss = float(latest["net_loss"].sum())

# Top 5 drivers by net loss (overall)
top_drivers = channel_product.head(5)

summary = []
summary.append(f"EXECUTIVE FRAUD RISK SUMMARY — {latest_dt.date()}")
summary.append("")
summary.append(f"• Total transactions: {day_txn:,}")
summary.append(f"• Fraud confirmed count: {day_fraud:,} ({day_rate:.3f}% fraud rate)")
summary.append(f"• Net fraud loss (loss - recoveries): ${day_net_loss:,.2f}")
summary.append("")
summary.append("Top Net Loss Drivers (Channel × Product):")
for _, r in top_drivers.iterrows():
    summary.append(f"• {r['channel']} / {r['product']}: Net Loss ${r['net_loss']:,.2f} | Fraud Rate {r['fraud_rate_pct']}% | Txns {int(r['txn_cnt']):,}")

exec_text = "\n".join(summary)

with open("outputs/executive_summary_text.txt", "w", encoding="utf-8") as f:
    f.write(exec_text)

print(exec_text)
print("\n✅ Saved: outputs/executive_summary_text.txt")


EXECUTIVE FRAUD RISK SUMMARY — 2025-11-30

• Total transactions: 708
• Fraud confirmed count: 12 (1.695% fraud rate)
• Net fraud loss (loss - recoveries): $349.90

Top Net Loss Drivers (Channel × Product):
• Card Not Present / Classic: Net Loss $24,791.46 | Fraud Rate 1.857% | Txns 51,807
• Card Not Present / Rewards: Net Loss $15,662.76 | Fraud Rate 1.876% | Txns 34,386
• Card Not Present / Premium: Net Loss $10,141.22 | Fraud Rate 1.908% | Txns 20,908
• Digital Wallet / Classic: Net Loss $7,853.91 | Fraud Rate 1.451% | Txns 20,255
• Card Present / Classic: Net Loss $6,100.82 | Fraud Rate 0.776% | Txns 31,439

✅ Saved: outputs/executive_summary_text.txt
